In [2]:
import pandas as pd
import numpy as np
import datetime as datetime
from datetime import timedelta
from faker import Faker

In [96]:
#Generated from file-metadata-generation.ipynb
user_files = pd.read_csv('Original File Information.csv')

In [17]:
user_files.head()

,file_name,file_extension,file_size,file_entropy
0,069.html,html,3002,4.600665
1,074.txt,txt,3898,4.698261
2,016.png,png,466245,7.932331
3,017.png,png,606390,7.929483
4,075.txt,txt,2845,4.487290


In [94]:
#Normal user Behavior Characteristics
#0-25 files accessed per day
#9am-5pm working hours

class NormalUser:
  def __init__(self, User_ID, IP_Address):
    self.User_ID = User_ID
    self.IP_Address = IP_Address
    self.maxDailyFiles = 25
    self.minDailyFiles = 0
    self.startTime = datetime.time(9,0)
    self.endTime = datetime.time(17,0)

In [95]:
#Abnormal/Anomalous User Behavior Characteristics
#10-1000 files accessed per day
#Working hours can be at any time of the day

class AbnormalUser:
  def __init__(self, User_ID, IP_Address):
    self.User_ID = User_ID
    self.IP_Address = IP_Address
    self.maxDailyFiles = 1000
    self.minDailyFiles = 10
    self.startTime = datetime.time(0,0)
    self.endTime = datetime.time(23,59)

In [33]:
class FileActions:
  def __init__(self, Sequence):
    self.Sequence = Sequence

In [47]:
class FileInfo:
  def __init__(self,file_name,extension,file_size,entropy, in_use, exists):
    self.file_name = file_name
    self.extension = extension
    self.file_size = file_size
    self.entropy = entropy
    self.in_use = in_use
    self.exists = exists

In [44]:
#Permissible File Actions
#Logic will be added in for probabilistic action selection, for e.g. delete actions should be infrequent

file_actions = [FileActions('Open->Close'),
                FileActions('Open->Modify->Close'),
                FileActions('Delete')
                ]

File information being obtained from Original File Information csv file

In [88]:
#File Objects being created from the original files dataframe
#RERUN EVERYTIME LOG CREATION STEP IS BEING RUN TO RESET TO ORIGINAL FILE PARAMS. 
#We must figure out a way to temporally maintain file characteristics

file_info = []
files_list = user_files.values.tolist()
for i in range(len(files_list)):
    file_metadata = files_list[i]
    file_obj = FileInfo(file_metadata[0],file_metadata[1],file_metadata[2],file_metadata[3],False,True)
    file_info.append(file_obj)

In [37]:
#USA IP ADDRESSES (normal)

normal_ips = ["205.175.106.235", "45.24.58.0", "172.120.61.61",
              "162.89.0.47", "50.73.157.178"]

In [38]:
#IP ADDRESSES FROM OUTSIDE OF THE US (abnormal)

abnormal_ips = ["218.107.132.66", "103.11.0.0", "100.43.64.7",
                "82.213.0.0", "103.48.198.141"]

In [39]:
#Random date generator

def generate_date():
    start_date = datetime.datetime(2023, 1, 1)
    end_date = datetime.datetime(2023, 3, 1)
    fake = Faker()
    random_datetime = fake.date_between(start_date, end_date)
    return random_datetime

In [40]:
#Time addition function

def time_plus(time, timedelta):
    start = datetime.datetime(
        2000, 1, 1,
        hour=time.hour, minute=time.minute, second=time.second)
    end = start + timedelta
    return end.time()

In [49]:
normal_user_data = pd.DataFrame(columns = ['user_id','user_ip','date','time','file','action','entropy'])

In [50]:
#Generate activity data for 1 normal user for 1 day
#Choose 1 IP address out of permissible ones
#Sample 0-25 files, generate actions for each file.
#Time of action is also sampled from permissible working hours, and each log entry is at intervals of 1-10 minutes
#Modifying a file equates to change in entropy
#Deleting a file makes it unavailable for future selection

ip = normal_ips[0]
user_id = "user1"
nu = NormalUser(user_id,ip)
num_files = np.random.randint(nu.minDailyFiles, nu.maxDailyFiles, 1)[0]
print(num_files)
random_date = generate_date()
random_hour = np.random.randint(nu.startTime.hour, nu.endTime.hour)
random_minute = np.random.randint(0,59)
random_second = np.random.randint(0,59)
random_time =datetime.time(hour=random_hour,minute=random_minute,second=random_second)
#print(random_time)
# print("random_date: ", random_date)
# print("random_time: ", random_time)
flen = len(file_info)
aclen = len(file_actions)
#files = np.random.randint(flen, size=num_files)
new_time = random_time
for i in range(num_files):
    #new_time = random_time
    file_random_number = np.random.randint(flen,size=1)[0]
    ac_random_number = np.random.randint(aclen,size=1)[0]
    #file_obj = file_info[files[i]]
    file_obj = file_info[file_random_number]
    #print(file_obj.file_name)
    ac_obj = file_actions[ac_random_number]
    print(file_obj.file_name)
    if file_obj.exists == False:
        print("Does not exist")
        continue
    
    #new_time = random_time
    if len(ac_obj.Sequence) > 1:
        num_actions = ac_obj.Sequence.split("->")
        #new_time = random_time
        for action in num_actions:
            row = []
            if action == "Open":
                file_obj.in_use = True
            else:
                file_obj.in_use = False
            if action == "Modify":
                change = np.random.uniform(-0.1,0.1)
                file_obj.entropy += change
            if action == "Delete":
                if file_obj.in_use == False and file_obj.exists == True:
                    file_obj.exists = False
                else:
                    break
            random_interval = np.random.randint(1,10)
            time_change = datetime.timedelta(minutes=random_interval)
            new_time = time_plus(new_time, time_change)
            #new_time = new_time + time_change
            row = []
            row.append(user_id)
            row.append(ip)
            row.append(random_date)
            row.append(new_time)
            row.append(file_obj.file_name)
            row.append(action)
            row.append(file_obj.entropy)
            normal_user_data = pd.concat([normal_user_data,pd.DataFrame([row],columns = ['user_id','user_ip','date','time','file','action','entropy'])], ignore_index=True)
            normal_user_data.reset_index()
            
    else:
        action = ac_obj.Sequence
        #new_time = random_time
        if action == "Open":
                file_obj.in_use = True
        else:
                file_obj.in_use = False


        if action == "Delete":
            if file_obj.in_use == False and file_obj.exists == True:
                file_obj.exists = False
            else:
                pass
        elif action == "Modify":
            change = np.random.uniform(-0.1,0.1)
            file_obj.entropy += change
            
        random_interval = np.random.randint(1,10)
        time_change = datetime.timedelta(minutes=random_interval)
        new_time = time_plus(new_time, time_change)
        #new_time = new_time + time_change
        row = []
        row.append(user_id)
        row.append(ip)
        row.append(random_date)
        row.append(new_time)
        row.append(file_obj.file_name)
        row.append(action)
        row.append(file_obj.entropy)
        normal_user_data = pd.concat([normal_user_data,pd.DataFrame([row],columns = ['user_id','user_ip','date','time','file','action','entropy'])], ignore_index=True)
        normal_user_data.reset_index()

9
005.jpg
073.txt
074.txt
083.doc
024.csv
098.doc.zip
024.csv
034.tsv
001.mp3


In [51]:
#User logs for 1 normal user for 1 day

normal_user_data

,user_id,user_ip,date,time,file,action,entropy
0,user1,205.175.106.235,2023-02-08,14:39:28,005.jpg,Delete,7.939269
1,user1,205.175.106.235,2023-02-08,14:45:28,073.txt,Delete,4.506627
2,user1,205.175.106.235,2023-02-08,14:46:28,074.txt,Open,4.698261
3,user1,205.175.106.235,2023-02-08,14:52:28,074.txt,Close,4.698261
4,user1,205.175.106.235,2023-02-08,14:54:28,083.doc,Open,4.73864
5,user1,205.175.106.235,2023-02-08,14:57:28,083.doc,Modify,4.780829
6,user1,205.175.106.235,2023-02-08,15:02:28,083.doc,Close,4.780829
7,user1,205.175.106.235,2023-02-08,15:08:28,024.csv,Open,4.569445
8,user1,205.175.106.235,2023-02-08,15:10:28,024.csv,Modify,4.490765
9,user1,205.175.106.235,2023-02-08,15:12:28,024.csv,Close,4.490765


In [89]:
encr_user_files = pd.read_csv('Encrypted File Information.csv')

In [90]:
abnormal_user_data = pd.DataFrame(columns = ['user_id','user_ip','date','time','file','action','entropy'])

In [91]:
#Generate activity data for 1 abnormal user for 1 day
#Choose 1 IP address out of permissible ones
#Sample 10-1000 files, generate actions for each file.
#Time of action is also sampled , and each log entry is at intervals of 1-10 minutes
#Modifying a file equates to encrypting the files, hence in the log,
#the file must change from normal extension to .wcry and entropy should change and be reflected accordingly.
#These details are taken from dataframe with encrypted files and pre computed entropies.
#Deleting a file makes it unavailable for future selection

ip = abnormal_ips[0]
user_id = "user2"
abnu = AbnormalUser(user_id,ip)
num_files = np.random.randint(abnu.minDailyFiles, abnu.maxDailyFiles, 1)[0]
print(num_files)
random_date = generate_date()
random_hour = np.random.randint(abnu.startTime.hour, abnu.endTime.hour)
random_minute = np.random.randint(0,59)
random_second = np.random.randint(0,59)
random_time =datetime.time(hour=random_hour,minute=random_minute,second=random_second)
#print(random_time)
# print("random_date: ", random_date)
# print("random_time: ", random_time)
flen = len(file_info)
aclen = len(file_actions)
#files = np.random.randint(flen, size=num_files)
new_time = random_time
for i in range(num_files):
    #new_time = random_time
    file_random_number = np.random.randint(flen,size=1)[0]
    ac_random_number = np.random.randint(aclen,size=1)[0]
    #file_obj = file_info[files[i]]
    file_obj = file_info[file_random_number]
    #print(file_obj.file_name)
    ac_obj = file_actions[ac_random_number]
    print(file_obj.file_name)
    if file_obj.exists == False:
        print("Does not exist")
        continue
    
    #new_time = random_time
    if len(ac_obj.Sequence) > 1:
        num_actions = ac_obj.Sequence.split("->")
        #new_time = random_time
        for action in num_actions:
            row = []
            if action == "Open":
                file_obj.in_use = True
            else:
                file_obj.in_use = False
            if action == "Modify":
                #ENCRYPTION
                if '.wcry' not in file_obj.file_name:
                    enc_name = file_obj.file_name +'.wcry'
                    enc_file_deets = encr_user_files.loc[encr_user_files['file_name']==enc_name]
                    file_obj.file_name = enc_file_deets['file_name'].values[0]
                    file_obj.extension = enc_file_deets['file_extension'].values[0]
                    file_obj.file_size = enc_file_deets['file_size'].values[0]
                    file_obj.entropy = enc_file_deets['file_entropy'].values[0]
#                 change = np.random.uniform(-0.1,0.1)
#                 file_obj.entropy += change
            if action == "Delete":
                if file_obj.in_use == False and file_obj.exists == True:
                    file_obj.exists = False
                else:
                    break
            random_interval = np.random.randint(1,10)
            time_change = datetime.timedelta(minutes=random_interval)
            new_time = time_plus(new_time, time_change)
            #new_time = new_time + time_change
            row = []
            row.append(user_id)
            row.append(ip)
            row.append(random_date)
            row.append(new_time)
            row.append(file_obj.file_name)
            row.append(action)
            row.append(file_obj.entropy)
            abnormal_user_data = pd.concat([abnormal_user_data,pd.DataFrame([row],columns = ['user_id','user_ip','date','time','file','action','entropy'])], ignore_index=True)
            abnormal_user_data.reset_index()
            
    else:
        action = ac_obj.Sequence
        #new_time = random_time
        if action == "Open":
                file_obj.in_use = True
        else:
                file_obj.in_use = False


        if action == "Delete":
            if file_obj.in_use == False and file_obj.exists == True:
                file_obj.exists = False
            else:
                pass
        elif action == "Modify":
            #ENCRYPTION
            if '.wcry' not in file_obj.file_name:
                enc_name = file_obj.file_name +'.wcry'
                enc_file_deets = encr_user_files.loc[encr_user_files['file_name']==enc_name]
                file_obj.file_name = enc_file_deets['file_name'].values[0]
                file_obj.extension = enc_file_deets['file_extension'].values[0]
                file_obj.file_size = enc_file_deets['file_size'].values[0]
                file_obj.entropy = enc_file_deets['file_entropy'].values[0]
#             change = np.random.uniform(-0.1,0.1)
#             file_obj.entropy += change
            
        random_interval = np.random.randint(1,10)
        time_change = datetime.timedelta(minutes=random_interval)
        new_time = time_plus(new_time, time_change)
        #new_time = new_time + time_change
        row = []
        row.append(user_id)
        row.append(ip)
        row.append(random_date)
        row.append(new_time)
        row.append(file_obj.file_name)
        row.append(action)
        row.append(file_obj.entropy)
        abnormal_user_data = pd.concat([abnormal_user_data,pd.DataFrame([row],columns = ['user_id','user_ip','date','time','file','action','entropy'])], ignore_index=True)
        abnormal_user_data.reset_index()

55
046.pdf
090.xml
031.xlsx
042.pdf
000.wav
056.pdf
044.pdf
051.pdf
041.pdf
093.mp4
019.gif
068.html
067.html
022.csv
092.mp4
001.mp3
025.csv
005.jpg
032.xlsx
006.jpg
094.mp4
012.jpg
037.pdf
031.xlsx
Does not exist
066.html
043.pdf
051.pdf
Does not exist
051.pdf
Does not exist
064.html
051.pdf
Does not exist
084.doc
052.pdf
015.png
064.html
Does not exist
024.csv
000.wav
Does not exist
022.csv
Does not exist
000.wav
Does not exist
025.csv.wcry
062.html
030.xlsx
049.pdf
064.html
Does not exist
028.csv
049.pdf.wcry
001.mp3.wcry
059.html
063.html
048.pdf
005.jpg
Does not exist
054.pdf
043.pdf
053.pdf
052.pdf
Does not exist
077.txt


In [93]:
#log activity for an anomalous user for 1 day

abnormal_user_data.head(20)

,user_id,user_ip,date,time,file,action,entropy
0,user2,218.107.132.66,2023-02-12,12:11:47,046.pdf,Delete,7.53863
1,user2,218.107.132.66,2023-02-12,12:15:47,090.xml,Open,4.597173
2,user2,218.107.132.66,2023-02-12,12:18:47,090.xml.wcry,Modify,7.929607
3,user2,218.107.132.66,2023-02-12,12:20:47,090.xml.wcry,Close,7.929607
4,user2,218.107.132.66,2023-02-12,12:21:47,031.xlsx,Delete,7.413686
5,user2,218.107.132.66,2023-02-12,12:29:47,042.pdf,Open,7.510405
6,user2,218.107.132.66,2023-02-12,12:35:47,042.pdf,Close,7.510405
7,user2,218.107.132.66,2023-02-12,12:36:47,000.wav,Delete,7.246942
8,user2,218.107.132.66,2023-02-12,12:39:47,056.pdf,Delete,7.442129
9,user2,218.107.132.66,2023-02-12,12:48:47,044.pdf,Delete,7.455804


Next steps will include:

1. Merging normal and anomalous behaviours to create combined log.
2. Changing action selection probabilities.
3. Figuring ways to maintain file parameters across time.